In [1]:
import pandas as pd

In [2]:
pontuacao_f1 = [25,18,15,12,10,8,6,4,2,1,1,0] # O penultimo numero é a melhor volta e o último numero é a pontuação do pole.
pontuacao_f1_sprint = [8,7,6,5,4,3,2,1,0,0,0,0]
jogo = "f1_2023"

if "f1_2022" in jogo:
    path_resultados = "Y://corrida_inscritos_2023.xls"
    opcao = 1
    if opcao == 1:
        path_output_csv = 'Y://pontuacao_corrida_dos_inscritos_2023.csv'
    elif opcao == 2:
        path_output_csv = 'Y://pontuacao_corrida_dos_membros_2023.csv'
elif "f1_2023" in jogo:
    path_resultados = "Y://corrida_inscritos_2024.xls"
    opcao = 2
    if opcao == 1:
        path_output_csv = 'Y://pontuacao_corrida_dos_inscritos_2024.csv'
    elif opcao == 2:
        path_output_csv = 'Y://pontuacao_corrida_dos_membros_2024.csv'
       

In [3]:
if opcao == 1:
    df_resultados = pd.read_excel(path_resultados, sheet_name="crossplay")
elif opcao == 2: 
    df_resultados = pd.read_excel(path_resultados, sheet_name="membros")
elif opcao == 3:
    pontuacao_f1 = [25,18,15,12,10,8,6,4,2,1,1,2]
    df_resultados = pd.read_excel(path_resultados, sheet_name="f2")


In [4]:
df_resultados.drop(["Index"], axis=1, inplace=True)

In [5]:
df_resultados

,Belgica-S1,Belgica,Holanda,Italia,Singapura,Japão,Catar-S1,Catar,Texas-S1,Texas,...,Jeddah,Melbourne,Suzuka,Shanghai-S1,Shanghai,Miami-S1,Miami,Imola,Monaco,Canadá
0,ZYON Leo Menon,ZYON Leo Menon,ZYON Leo Menon,Muka,samuel_freitas26,ZYON Patrick,Alienobserv,ZYON Leo Menon,DarkLightBR,ZYON Leo Menon,...,ZYON Leo Menon,BrPB Vitor,Gabriel Quintino,ZYON Patrick,ZYON Patrick,ZYON Leo Menon,ZYON Leo Menon,ZYON Leo Menon,ZYON Leo Menon,BrPB Vitor
1,samuel_freitas26,alissonzeira,ZYON Patrick,crazy-mars245,LucasBR21,LucasBR21,O_peralta99,LucasBR21,ZYON Leo Menon,BrPB Vitor,...,BrPB Vitor,ZYON Sucla,BrPB Vitor,ZYON Leo Menon,ZYON Leo Menon,BrPB Vitor,Spidey,LucasBR21,LucasBR21,Juankzer
2,LucasBR21,Kuro,Kuro,ZYON Leo Menon,crazy-mars245,ZYON Leo Menon,LucasBR21,Alienobserv,BrPB Vitor,LucasBR21,...,LucasBR21,LucasBR21,samuel_freitas26,LucasBR21,ZYON Sucla,Nickolas Machado,LucasBR21,Nickolas Machado,ZYON Sucla,LucasBR21
3,Bill25binho,Zedatapioca,O_peralta99,samuel_freitas26,Savio-23BRA,Kuro,alissonzeira,Cristiano Roque,samuel_freitas26,Leozin_CRF,...,alissonzeira,ZYON Patrick,ZYON Leo Menon,samuel_freitas26,BrPB Vitor,Spidey,BrPB Vitor,ZYON Patrick,Nickolas Machado,ZYON Sucla
4,Savio-23BRA,ZYON Patrick,samuel_freitas26,Dinner,O_peralta99,O_peralta99,Fernando Vignola,TonyBR80,Leozin_CRF,DarkLightBR,...,samuel_freitas26,ZYON Leo Menon,LucasBR21,silvawolfw,nahtanN,LucasBR21,ZYON Sucla,nahtaN,Savio-23BRA,O_peralta99
5,ZYON Patrick,O_peralta99,Dinner,Kuro,alissonzeira,Alienobserv,Cristiano Roque,ZYON Sucla,ZYON Sucla,samuel_freitas26,...,Zedatapioca,samuel_freitas26,O_peralta99,Samuel Pereira,Alienobserv,Alienobserv,Felipe Braule,Alienobserv,Zedatapioca,Alienobserv
6,Muka,Savio-23BRA,Alienobserv,Cristiano Roque,Kuro,alissonzeira,Ailton4514,Zedatapioca,O_peralta99,O_peralta99,...,Alienobserv,alissonzeira,Zedatapioca,BrPB Vitor,bestzinhoxd,nahtaN,Nickolas Machado,BrPB Vitor,O_peralta99,Nahtan
7,ZYON Sucla,LucasBR21,matheus_fw02,Johnyangry,ZYON Patrick,TonyBR80,Zedatapioca,samuel_freitas26,LucasBR21,ZYON Sucla,...,Dragoonie Skywere,O_peralta99,Alienobserv,bestzinhoxd,LucasBR21,Savio-23BRA,O_peralta99,ZYON Sucla,Samuel Pereira,Samuel Freitas
8,Zedatapioca,Ailton4514,LucasBR21,TonyBR80,Fernando Vignola,samuel_freitas26,ZYON Leo Menon,alissonzeira,Ailton4514,Cristiano Roque,...,ZYON Sucla,AlienObserv,silvawolfw,Tomasglopes,Dragoonie Skywere,Samuel Pereira,nahtaN,O_peralta99,alissonzeira,Dragoonie
9,O_peralta99,Fernando Vignola,Muka,matheus_fw02,Muka,Cristiano Roque,Savio-23BRA,O_peralta99,Muka,Alienobserv,...,Bill25binho,Cristiano Roque,The_Fire_King_BR,Dragoonie Skywere,Tomasglopes,O_peralta99,Alienobserv,TheFireKing21,DNF,TheFireKing


In [6]:
df_pontos = pd.DataFrame(columns=["Piloto", "Pontos"])

In [7]:
for etapa in list(df_resultados.columns):
    df_etapa = pd.DataFrame(columns=["Piloto", "Pontos"])
    if (etapa.endswith("-S1")) or (etapa.endswith("-S2")): #código para Sprint (Nome da etapa seria exemplo Bahrain-S)
        df_etapa["Pontos"] = pontuacao_f1_sprint
    else:
        df_etapa["Pontos"] = pontuacao_f1
    df_etapa["Piloto"] = [driver.upper() for driver in df_resultados[etapa].values]
    df_pontos = pd.concat([df_pontos, df_etapa], ignore_index=True)

#### Gerando pontuação

In [8]:
try:
    df_pontuacao = df_pontos.groupby("Piloto").sum().sort_values(by="Pontos", ascending=False).drop("DNF", axis=0)
    df_pontuacao["Lider"] = df_pontuacao["Pontos"].values[0]
    df_pontuacao["Diferenca"] = df_pontuacao["Pontos"] - df_pontuacao["Lider"]
    df_pontuacao.drop(["Lider"], axis=1, inplace=True)
    df_pontuacao
except KeyError: #nao achou dnf
    df_pontuacao = df_pontos.groupby("Piloto").sum().sort_values(by="Pontos", ascending=False)
    df_pontuacao["Lider"] = df_pontuacao["Pontos"].values[0]
    df_pontuacao["Diferenca"] = df_pontuacao["Pontos"] - df_pontuacao["Lider"]
    df_pontuacao.drop(["Lider"], axis=1, inplace=True)
    df_pontuacao

#### Gera pontuação extendida

In [9]:
for etapa in list(df_resultados.columns):
    pilotos = [driver.upper() for driver in df_resultados[etapa].values]
    df_pontuacao[etapa] = 0
    if (etapa.endswith("-S1")) or (etapa.endswith("-S2")):
        for eachDriver, points in zip(pilotos, pontuacao_f1_sprint):
            if eachDriver != 'DNF':
                if df_pontuacao.at[eachDriver, etapa] == 0:
                    df_pontuacao.at[eachDriver, etapa] = points
                else:
                    df_pontuacao.at[eachDriver, etapa] += points
    else:
        for eachDriver, points in zip(pilotos, pontuacao_f1):
            if eachDriver != 'DNF':
                if df_pontuacao.at[eachDriver, etapa] == 0:
                    df_pontuacao.at[eachDriver, etapa] = points
                else:
                    df_pontuacao.at[eachDriver, etapa] += points

In [10]:
# Reposicionando as colunas
novasColunas = list(df_resultados.columns)
novasColunas.append("Pontos")
novasColunas.append("Diferenca")
df_pontuacao = df_pontuacao[novasColunas]
df_pontuacao[:50]

,Belgica-S1,Belgica,Holanda,Italia,Singapura,Japão,Catar-S1,Catar,Texas-S1,Texas,...,Suzuka,Shanghai-S1,Shanghai,Miami-S1,Miami,Imola,Monaco,Canadá,Pontos,Diferenca
Piloto,,,,,,,,,,,,,,,,,,,,,
ZYON LEO MENON,8,25,25,15,0,15,0,26,7,25,...,12,7,18,8,25,25,25,0,336,0
LUCASBR21,6,4,2,0,18,18,6,18,1,15,...,10,6,4,4,15,18,18,15,308,-28
ZYON PATRICK,3,10,19,0,5,26,0,0,0,0,...,0,8,25,0,0,13,0,0,195,-141
BRPB VITOR,0,0,0,0,0,0,0,0,6,18,...,18,2,13,7,12,6,0,25,151,-185
SAMUEL_FREITAS26,7,0,10,12,25,2,0,4,5,8,...,15,5,0,0,0,0,0,0,147,-189
ZYON SUCLA,1,0,0,0,0,0,0,8,3,4,...,0,0,15,0,10,4,15,12,138,-198
O_PERALTA99,0,8,12,0,10,10,7,1,2,6,...,8,0,0,0,4,2,6,10,136,-200
ALIENOBSERV,0,0,6,0,0,8,8,15,0,1,...,4,0,8,3,1,8,0,8,88,-248
ALISSONZEIRA,0,18,0,0,8,6,5,2,0,0,...,0,0,0,0,0,0,2,0,69,-267


In [11]:
df_pontuacao[51:]

,Belgica-S1,Belgica,Holanda,Italia,Singapura,Japão,Catar-S1,Catar,Texas-S1,Texas,...,Suzuka,Shanghai-S1,Shanghai,Miami-S1,Miami,Imola,Monaco,Canadá,Pontos,Diferenca
Piloto,,,,,,,,,,,,,,,,,,,,,


In [12]:
df_pontuacao[["Pontos", "Diferenca"]][:50]

,Pontos,Diferenca
Piloto,,
ZYON LEO MENON,336,0
LUCASBR21,308,-28
ZYON PATRICK,195,-141
BRPB VITOR,151,-185
SAMUEL_FREITAS26,147,-189
ZYON SUCLA,138,-198
O_PERALTA99,136,-200
ALIENOBSERV,88,-248
ALISSONZEIRA,69,-267


In [13]:
df_pontuacao[["Pontos", "Diferenca"]][51:]

,Pontos,Diferenca
Piloto,,


In [14]:
if opcao == 1:
    df_pontuacao.to_csv(path_output_csv)
    print("Pontuação da Corrida dos Inscritos exportada com sucesso...")
elif opcao == 2:
    df_pontuacao.to_csv(path_output_csv)
    print("Pontuação da Corrida dos Membros exportada com sucesso...")

Pontuação da Corrida dos Membros exportada com sucesso...


#### Exportando a tabela em HTML

In [15]:
from  datetime import datetime

# Criar um DataFrame de exemplo (substitua isso com seus próprios dados)
numero_de_linhas = df_pontuacao.shape[0]
numero_de_colunas = df_pontuacao.shape[1]

html_table = df_pontuacao.to_html(index=True)

if opcao == 1:
    title = "Tabela Corrida dos Inscritos"
elif opcao == 2:
    title = "Tabela Corrida dos Membros"

df_pontuacao = df_pontuacao.reset_index()
df_pontuacao.index = range(1, numero_de_linhas + 1)
html_table = df_pontuacao.to_html(index=False, escape=False)
data_hoje = (datetime.now()).strftime("%d/%m/%Y %H:%M:%S")

# Criar tabela HTML com CSS incorporado
html_code = f'''
<html>
<head>
    <meta charset="latin-1"> 
    <link rel="stylesheet" type="text/css" href="styles.css">
</head>
<body>
    <h4>Atualizado em: {data_hoje}</h4>
    {html_table}
</body>
</html>
'''

# Escrever o HTML para um arquivo
if opcao == 1:
    nome_arquivo = 'index_corrida_inscritos.html'
else:
    nome_arquivo = 'index_corrida_membros.html'
    
with open(f'{nome_arquivo}', 'w') as file:
    file.write(html_code)

print(f"Arquivo HTML exportado com sucesso: {nome_arquivo}.html")

Arquivo HTML exportado com sucesso: index_corrida_membros.html.html


### Calculo da pontuação mensal para o gift card  
Insira o intervalo das etapas equivalente ao mês que deseja calcular a pontuação.  
Exemplo: Se as etapas 3 e 4 forem as etapas realizadas no determinado mês, coloque entre colchetes '[2, 3]'

In [16]:
if opcao == 2:
    try:
        mes = {1: "Janeiro", 2: "Fevereiro", 3: "Março", 4:"Abril",
               5: "Maio", 6:"Junho", 7: "Julho", 8: "Agosto",
               9: "Setembro", 10: "Outubro", 11: "Novembro", 12: "Dezembro"}
        etapa_inicio = 26
        etapa_fim = 31
        mes_code = 6
        df_pontos = pd.DataFrame(columns=["Piloto", "Pontos"])

        for etapa in list(df_resultados.columns[etapa_inicio-1:etapa_fim]):
            df_etapa = pd.DataFrame(columns=["Piloto", "Pontos"])
            if etapa.endswith("-S1"): #código para Sprint (Nome da etapa seria exemplo Bahrain-S)
                df_etapa["Pontos"] = pontuacao_f1_sprint
            else:
                df_etapa["Pontos"] = pontuacao_f1
            df_etapa["Piloto"] = [driver.upper() for driver in df_resultados[etapa].values]
            df_pontos = pd.concat([df_pontos, df_etapa], ignore_index=True)

        try:
            df_pontuacao = df_pontos.groupby("Piloto").sum().sort_values(by="Pontos", ascending=False).drop("DNF", axis=0)
            df_pontuacao["Lider"] = df_pontuacao["Pontos"].values[0]
            df_pontuacao["Diferenca"] = df_pontuacao["Pontos"] - df_pontuacao["Lider"]
            df_pontuacao.drop(["Lider"], axis=1, inplace=True)
            df_pontuacao
        except KeyError: #nao achou dnf
            df_pontuacao = df_pontos.groupby("Piloto").sum().sort_values(by="Pontos", ascending=False)
            df_pontuacao["Lider"] = df_pontuacao["Pontos"].values[0]
            df_pontuacao["Diferenca"] = df_pontuacao["Pontos"] - df_pontuacao["Lider"]
            df_pontuacao.drop(["Lider"], axis=1, inplace=True)
            df_pontuacao

        for etapa in list(df_resultados.columns[etapa_inicio-1:etapa_fim]):
            pilotos = [driver.upper() for driver in df_resultados[etapa].values]
            df_pontuacao[etapa] = 0
            if (etapa.endswith("-S1")) or (etapa.endswith("-S2")):
                for eachDriver, points in zip(pilotos, pontuacao_f1_sprint):
                    if eachDriver != 'DNF':
                        if df_pontuacao.at[eachDriver, etapa] == 0:
                            df_pontuacao.at[eachDriver, etapa] = points
                        else:
                            df_pontuacao.at[eachDriver, etapa] += points
            else:
                for eachDriver, points in zip(pilotos, pontuacao_f1):
                    if eachDriver != 'DNF':
                        if df_pontuacao.at[eachDriver, etapa] == 0:
                            df_pontuacao.at[eachDriver, etapa] = points
                        else:
                            df_pontuacao.at[eachDriver, etapa] += points

        # Reposicionando as colunas
        novasColunas = list(df_resultados.columns[etapa_inicio-1:etapa_fim])
        novasColunas.append("Pontos")
        novasColunas.append("Diferenca")
        df_pontuacao = df_pontuacao[novasColunas]
        print(f"Pontuação dos membros do mês de {mes.get(mes_code)}")
        print(df_pontuacao[:50])
    except IndexError:
        print("Nenhuma etapa ainda no mês de {}".format(mes.get(mes_code)))
else:
    print("Pontuação mensal só é válida para a corrida dos membros!")

Pontuação dos membros do mês de Junho
                Canadá Pontos Diferenca
Piloto                                 
BRPB VITOR          25     25         0
JUANKZER            18     18        -7
LUCASBR21           15     15       -10
ZYON SUCLA          12     12       -13
O_PERALTA99         10     10       -15
ALIENOBSERV          8      8       -17
NAHTAN               6      6       -19
SAMUEL FREITAS       4      4       -21
DRAGOONIE            2      2       -23
THEFIREKING          1      1       -24


#### Exportar HTML para corrida dos membros

In [17]:
if opcao == 2:
    from  datetime import datetime

    # Criar um DataFrame de exemplo (substitua isso com seus próprios dados)
    numero_de_linhas = df_pontuacao.shape[0]
    numero_de_colunas = df_pontuacao.shape[1]

    html_table = df_pontuacao.to_html(index=True)

    if opcao == 1:
        title = "Tabela Corrida dos Inscritos"
    elif opcao == 2:
        title = "Tabela Corrida dos Membros"

    df_pontuacao = df_pontuacao.reset_index()
    df_pontuacao.index = range(1, numero_de_linhas + 1)
    html_table = df_pontuacao.to_html(index=False, escape=False)
    data_hoje = (datetime.now()).strftime("%d/%m/%Y %H:%M:%S")

    # Criar tabela HTML com CSS incorporado
    html_code = f'''
    <html>
    <head>
        <meta charset="latin-1">
         <link rel="stylesheet" type="text/css" href="styles.css">
    </head>
    <body>
        <h2>Pontuacao dos membros do mes de {mes.get(mes_code)}
        <h4>Atualizado em: {data_hoje}</h4>
        {html_table}
    </body>
    </html>
    '''

    # Escrever o HTML para um arquivo
    if opcao == 2:
        nome_arquivo = 'index_gift_mensal.html'

    with open(f'{nome_arquivo}', 'w') as file:
        file.write(html_code)

    print("Arquivo HTML exportado com sucesso: index_gift_mensal.html")

Arquivo HTML exportado com sucesso: index_gift_mensal.html


#### Atualiza index com a corrida mais recente

In [18]:
corrida_inscritos_mais_recente_id = "EYD7CyUtRKM"

In [19]:
html_code = f"""
<!DOCTYPE html>
<html>
  <head>
	<meta charset="latin-1">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Corrida dos Inscritos 2024</title>
	<link rel="stylesheet" type="text/css" href="styles.css">
  </head>
  <body>
  
  <header>
    <h1><a href='index.html'>Corrida dos Inscritos 2024</a></h1>
  </header>
	
    <!-- O <iframe> (e o player de vídeo) substituirão esta tag <div>. -->
    <div id="player"></div>

    <script>
      var tag = document.createElement('script');
      tag.src = "https://www.youtube.com/iframe_api";
      var firstScriptTag = document.getElementsByTagName('script')[0];
      firstScriptTag.parentNode.insertBefore(tag, firstScriptTag);

      var player;
      function onYouTubeIframeAPIReady() {{
        player = new YT.Player('player', {{
          height: '100%',
          width: '100%',
          videoId: '{corrida_inscritos_mais_recente_id}',
		  playerVars: {{ 'autoplay': 1, 'controls': 1 }},
          events: {{
            'onReady': onPlayerReady,   
          }}
        }});
		player.setPlaybackQuality('hd720')
      }}

      function onPlayerReady(event) {{
        event.target.playVideo();
      }}

      var done = false;
      function onPlayerStateChange(event) {{
        if (event.data == YT.PlayerState.PLAYING && !done) {{
          done = true;
        }}
      }}

      function stopVideo() {{
        player.stopVideo();
      }}
	  
	  
    </script>
  </body>
</html>
"""

with open(f'ultima_corrida_inscritos.html', 'w') as file:
        file.write(f'{html_code}')

#### Ligar o server em Python 

In [20]:
#!python -m http.server